In [1]:
!pip install –upgrade pip

ERROR: Invalid requirement: '–upgrade'


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.0.17 requires srsly<3.0.0,>=2.4.0, but you have srsly 1.0.5 which is incompatible.
spacy 3.3.0 requires srsly<3.0.0,>=2.4.3, but you have srsly 1.0.5 which is incompatible.


  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5


spacy-transformers 1.1.6 requires srsly<3.0.0,>=2.4.0, but you have srsly 1.0.5 which is incompatible.


In [3]:
!pip install -U spacy
!python -m spacy download en_core_web_trf
import en_core_web_trf
nlp = en_core_web_trf.load()

  Using cached srsly-2.4.3-cp38-cp38-win_amd64.whl (448 kB)
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5


2022-06-03 15:36:59.204435: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-06-03 15:36:59.204486: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Traceback (most recent call last):
  File "C:\Users\msi\anaconda3\lib\runpy.py", line 185, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "C:\Users\msi\anaconda3\lib\runpy.py", line 144, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "C:\Users\msi\anaconda3\lib\runpy.py", line 111, in _get_module_details
    __import__(pkg_name)
  File "C:\Users\msi\anaconda3\lib\site-packages\spacy\__init__.py", line 15, in <module>
    from .cli.info import info  # noqa: F401
  File "C:\Users\msi\anaconda3\lib\site-packages\spacy\cli\__init__.py", line 17, in <module>
    from .debug

ImportError: cannot import name 'soft_unicode' from 'markupsafe' (C:\Users\msi\anaconda3\lib\site-packages\markupsafe\__init__.py)

In [1]:
import spacy
nlp = spacy.load("en_core_web_trf")
doc = nlp("Madison is colder than St.louis but warmer than Butler County, Also Bayer has installed a new Pipeline in WI ")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Madison 0 7 GPE
St.louis 23 31 GPE
Butler County 48 61 GPE
Bayer 68 73 ORG
Pipeline 94 102 FAC
WI 106 108 GPE


C:\Users\msi\anaconda3\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [ ]:
pip install geonamescache

In [33]:
import geonamescache

gc = geonamescache.GeonamesCache()

# gets nested dictionary for countries
countries = gc.get_countries()

# gets nested dictionary for cities
cities = gc.get_cities()

In [34]:
def gen_dict_extract(var, key):
    if isinstance(var, dict):
        for k, v in var.items():
            if k == key:
                yield v
            if isinstance(v, (dict, list)):
                yield from gen_dict_extract(v, key)
    elif isinstance(var, list):
        for d in var:
            yield from gen_dict_extract(d, key)


In [35]:
cities = [*gen_dict_extract(cities, 'name')]
countries = [*gen_dict_extract(countries, 'name')]

In [36]:
nlp = spacy.load("en_core_web_trf")
doc= nlp(" Sac & Story county IA,West Central Indiana,jackson county WI,Pierce county WI,Milwaukee WI, Apple, bayer,Kansas, Southern Minnesota, Bloomfield Indiana, Saline county MO, Central IL, Springfield IL, Coast of Maine,Madison,India,United States of America,Bangladesh,Brazil")
for ent in doc.ents:
    if ent.label_ == 'GPE':
        if ent.text in countries:
            print(f"Country : {ent.text}")
        elif ent.text in cities:
            print(f"City : {ent.text}")
        else:
            print(f"Other GPE : {ent.text}")
    
         


Other GPE : Sac
Other GPE : West Central Indiana
Other GPE : jackson county
Other GPE : WI
Other GPE : Pierce county WI
City : Milwaukee
Other GPE : WI
Other GPE : Apple
Other GPE : bayer
Other GPE : Kansas
Other GPE : Minnesota
City : Bloomfield
Other GPE : Indiana
Other GPE : Saline county MO
Other GPE : Central IL
City : Springfield
Other GPE : IL
City : Madison
Country : India
Other GPE : United States of America
Country : Bangladesh
Country : Brazil


In [1]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm

In [2]:
TRAIN_DATA = [
    ('Pierce County', {
        'entities': [(0, 12, 'GPE')]
    }),
     ('Jackson County', {
        'entities': [(0, 13, 'GPE')]
    }),
    ('I like Madison and IL.', {
        'entities': [(7, 14, 'GPE'), (19, 21, 'GPE')] 
    }),
     ('I like Arizona and IA.', {
        'entities': [(7, 14, 'GPE'), (19, 21, 'GPE')] 
         })
    ]

In [3]:
model = None

n_iter=100

In [4]:
if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

Created blank 'en' model


In [5]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner',last=True)
else:
    ner = nlp.get_pipe('ner')

In [ ]:
from spacy.training.example import Example
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for batch in spacy.util.minibatch(TRAIN_DATA, size=5):
          for text, annotations in batch:
        # create Example
           doc = nlp.make_doc(text)
           example = Example.from_dict(doc, annotations)
        # Update the model
           nlp.update([example], losses=losses, drop=0.5,sgd=optimizer)
                
        print(losses)

In [25]:
import pandas as pd
df= pd.read_csv(r"C:\Users\msi\uscounties.csv", encoding= 'latin-1')

In [26]:
df.head()

,County,county_ascii,County_full,county_fips,state_id,State_name,lat,lng,population
0,Los Angeles,Los Angeles,Los Angeles County,6037,CA,California,34.3209,-118.2247,10040682
1,Cook,Cook,Cook County,17031,IL,Illinois,41.8401,-87.8168,5169517
2,Harris,Harris,Harris County,48201,TX,Texas,29.8578,-95.3936,4680609
3,Maricopa,Maricopa,Maricopa County,4013,AZ,Arizona,33.3490,-112.4915,4412779
4,San Diego,San Diego,San Diego County,6073,CA,California,33.0343,-116.7350,3323970


In [27]:
df['County_full']

0       Los Angeles County
1              Cook County
2            Harris County
3          Maricopa County
4         San Diego County
               ...        
3138        Harding County
3139      McPherson County
3140         Kenedy County
3141           King County
3142         Loving County
Name: County_full, Length: 3143, dtype: object

In [37]:
import spacy
nlp = spacy.load("en_core_web_trf")
doc= nlp(" Sac & Story county IA,San Diego County,West Central Indiana,jackson county WI,Pierce county WI,Milwaukee WI, Apple, bayer,Kansas, Southern Minnesota, Bloomfield Indiana, Saline county MO, Central IL, Springfield IL, Coast of Maine,Madison,India,United States of America,Bangladesh,Brazil")
for ent in doc.ents:
    if ent.label_ == 'GPE':
        if ent.text in df['County_full']:
            print(f"County : {ent.text}")
        elif ent.text in df:
            print(f"State : {ent.text}")
        else:
            print(f"Other GPE : {ent.text}")

Other GPE : Sac
Other GPE : IA
Other GPE : San Diego County
Other GPE : West Central Indiana
Other GPE : jackson county
Other GPE : WI
Other GPE : Pierce county WI
Other GPE : Milwaukee
Other GPE : WI
Other GPE : Apple
Other GPE : bayer
Other GPE : Kansas
Other GPE : Southern Minnesota
Other GPE : Bloomfield
Other GPE : Indiana
Other GPE : Saline county
Other GPE : MO
Other GPE : Central IL
Other GPE : Springfield
Other GPE : IL
Other GPE : Madison
Other GPE : India
Other GPE : United States of America
Other GPE : Bangladesh
Other GPE : Brazil
